In [ ]:
from ultralytics import YOLO
import torch
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import mimetypes

In [ ]:
path = "to_YOLO_MODEL"

In [ ]:
def test_model():
    # Check if GPU is available and set the device
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Using device: {device}")

    model = YOLO(f"{path}")

    source = "test_images/images"

    results = model.predict(source, save_txt=True, save=True)

    for r in results:
        print(r.boxes)

In [ ]:
if __name__=="__main__":
    test_model()

In [ ]:
def show_mask(mask, ax, random_color=False):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        color = np.array([30/255, 144/255, 255/255, 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)
    
def show_points(coords, labels, ax, marker_size=375):
    pos_points = coords[labels==1]
    neg_points = coords[labels==0]
    ax.scatter(pos_points[:, 0], pos_points[:, 1], color='green', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)
    ax.scatter(neg_points[:, 0], neg_points[:, 1], color='red', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)   
    
def show_box(box, ax):
    x0, y0 = box[0], box[1]
    w, h = box[2] - box[0], box[3] - box[1]
    ax.add_patch(plt.Rectangle((x0, y0), w, h, edgecolor='green', facecolor=(0,0,0,0), lw=2)) 

In [ ]:
# Define the base directory where the results are saved
base_dir = "runs/detect"

# Get a list of all subdirectories in the base directory
sub_dirs = [os.path.join(base_dir, d) for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d))]

# Sort the subdirectories by creation time
sub_dirs.sort(key=lambda x: os.path.getmtime(x), reverse=True)

# The first subdirectory in the sorted list is the latest directory
latest_dir = sub_dirs[0]

# Define the subdirectory where the .txt files are saved
sub_dir = "labels"

# Construct the full path to the .txt files
txt_files_path = os.path.join(latest_dir, sub_dir)

    
print(txt_files_path)

In [ ]:
# Check if the directory exists
if os.path.isdir(txt_files_path):
    # Get a list of all files in the directory
    txt_files = sorted(os.listdir(txt_files_path))
    print(txt_files)
else:
    print(f"The directory {txt_files_path} does not exist.")

In [ ]:
# Iterate over the sorted lists
labels = []
order = []

for filename in txt_files:
    print(filename)
    
    # Get the number from file name
    current = (int(filename.split('.')[0]))

    # Check if the file is a txt file
    mime_type, _ = mimetypes.guess_type(filename)
    if mime_type in ['text/plain']:
        # print('It is a text file')
        
        # Construct the full path to the file
        full_path = os.path.join(txt_files_path, filename)
        print(full_path)

        # Get class_id from the id file
        with open(full_path, 'r') as f:
            lines = f.readlines()
            print(lines)

        # Get the annotations from the annotation file
        for line in lines:
            other_parts = line.split()
            annotations = other_parts[1:]
            labels.append(annotations)
            order.append(current)

In [ ]:
print(labels)
print(order)

In [ ]:
def display_and_convert_images(path):
    # Get a list of all files in the directory
    files = os.listdir(path)

    # Determine the number of images
    num_images = len(files)

    # Determine subplot size
    num_cols = 4
    if (num_images + 1) % num_cols == 0:
        num_rows = (num_images + 1) // num_cols
    else:
        num_rows = (num_images + 1) // num_cols + 1

    # Create a new figure
    plt.figure(figsize=(10 * num_cols, 10 * num_rows))

    images = []
    images_paths = []

    # Iterate over all files in the directory
    for i, filename in enumerate(files, start=1):
        # Construct the full path to the image file
        image_path = os.path.join(path, filename)

        # Read the image
        image = cv2.imread(image_path)

        # Convert the image from BGR to RGB
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Add a subplot for the current image
        plt.subplot(num_rows, num_cols, i)

        # Display the image
        plt.imshow(image)
        plt.axis('off')
        images.append(image)
        images_paths.append(filename)
        

    # Show all images
    plt.show()

    return images, images_paths

In [ ]:
path = "path_to_images"
images, images_paths = display_and_convert_images(path)

In [ ]:
def convert_bboxes(bboxes, image_height, image_width):

    input_boxes = []
    input_points = []
    
    for bbox in bboxes:
        # Extract the normalized bounding box values
        x_center, y_center, box_width, box_height = bbox
    
        # Unnormalize the values to get pixel coordinates
        x_center = x_center * image_width
        y_center = y_center * image_height
        box_width = box_width * image_width
        box_height = box_height * image_height
    
        # Calculate the top-left (x_min, y_min) and bottom-right (x_max, y_max) corners
        x_min = x_center - (box_width / 2)
        y_min = y_center - (box_height / 2)
        x_max = x_center + (box_width / 2)
        y_max = y_center + (box_height / 2)
    
        # Append the converted bounding box to the list
        input_boxes.append([x_min, y_min, x_max, y_max])
        input_points.append([x_center, y_center])
    
        input_box = np.array(input_boxes)
        input_point = np.array(input_points)
        input_label = np.array([1]*len(input_boxes))
    


    return input_box, input_point, input_label

In [ ]:
import sys
sys.path.append("..")
from segment_anything import sam_model_registry, SamPredictor

sam_checkpoint = "path_to_SAM_checkpoint"  # Check SAM jupyter Notebook for the checkpoint instructions
model_type = "vit_h"

device = "cuda"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

predictor = SamPredictor(sam)


In [ ]:
# Convert string values to float
labels = [[float(coord) for coord in bbox] for bbox in labels]

i = 0

# Initialize a dictionary to collect bounding boxes per image
image_to_boxes = {}

# Iterate over images and order simultaneously
for idx in range(len(order)):
    current_idx = order[idx]
    
    # Get the number from file name
    filename = int(images_paths[current_idx].split('.')[0])
    
    # Get the labels for the current image
    image_labels = labels[i]
    
    # If the current image is already in the dictionary, append the new bounding box
    if current_idx in image_to_boxes:
        image_to_boxes[current_idx].append(image_labels)
    else:
        # Otherwise, create a new entry for this image
        image_to_boxes[current_idx] = [image_labels]
    
    # Move to the next label
    i += 1

# Now you have all bounding boxes for each image collected.
# Let's iterate over the images and process all bounding boxes at once
for idx, boxes in image_to_boxes.items():
    image = images[idx]

    # Assuming image_height and image_width are defined
    image_height, image_width, _ = image.shape
    
    # Convert all boxes for the current image
    input_boxes, input_points, input_labels = convert_bboxes(boxes, image_height, image_width)

    # Create masks and process the image with all bounding boxes
    predictor.set_image(image)
    
    # For multiple boxes
    ib = torch.tensor(input_boxes, device=predictor.device)
    
    transformed_boxes = predictor.transform.apply_boxes_torch(ib, image.shape[:2])
    
    masks, _, _ = predictor.predict_torch(
    point_coords=None,
    point_labels=None,
    boxes=transformed_boxes,
    multimask_output=False,
    )
    
    # Display the image
    plt.imshow(image)
    plt.axis('off')
    for mask in masks:
        show_mask(mask.cpu().numpy(), plt.gca(), random_color=True)
    for box in ib:
        show_box(box.cpu().numpy(), plt.gca())
    show_points(input_points, input_labels, plt.gca())
    plt.figure(figsize=(10,10))
    plt.show() 
    